## Benchmark for measuring performance of engineered features and models

Prepare data for later analysis

In [1]:
import sys
sys.path.insert(0, '../../bosch_helper')
from bosch_helper import *

%matplotlib inline

### Reload data

In [2]:
x = pd.read_hdf('numeric_all.hdf', 'x')

In [3]:
x.head()

L0_S0_F0  L0_S0_F10  L0_S0_F12  L0_S0_F14  L0_S0_F16  L0_S0_F18  \
      Id                                                                        
test  510783    -0.180     -0.066      0.022      0.128     -0.204     -0.147   
train 651542    -0.180     -0.021      0.022      0.168     -0.220     -0.163   
      108193    -0.193     -0.021      0.030      0.208     -0.266     -0.078   
test  430412    -0.153     -0.066      0.022      0.168     -0.056      0.029   
      443497    -0.297     -0.021      0.052      0.208     -0.128     -0.051   

              L0_S0_F2  L0_S0_F20  L0_S0_F22  L0_S0_F4    ...     \
      Id                                                  ...      
test  510783    -0.213     -0.340     -0.340     0.367    ...      
train 651542    -0.250     -0.340     -0.340     0.367    ...      
      108193    -0.272     -0.340     -0.340     0.385    ...      
test  430412    -0.235     -0.338     -0.338     0.385    ...      
      443497    -0.287     -0.340     -0.340     0.367    ...      

              L3_S50_F4253  L3_S51_F4256  L3_S51_F4258  L3_S51_F4260  \
      Id                                                               
test  510783           NaN           NaN           NaN           NaN   
train 651542           NaN           NaN           NaN           NaN   
      108193           NaN           NaN           NaN           NaN   
test  430412           NaN           NaN           NaN           NaN   
      443497           NaN           NaN           NaN           NaN   

              L3_S51_F4262  Response  station_start  station_end  time_start  \
      Id                                                                       
test  510783           NaN       NaN            0.0         52.0        0.00   
train 651542           NaN       0.0            0.0         52.0        0.00   
      108193           NaN       0.0            0.0         52.0        0.01   
test  430412           NaN       NaN            0.0         52.0        0.01   
      443497           NaN       NaN            0.0         52.0        0.01   

              time_end  
      Id                
test  510783      1.61  
train 651542      1.53  
      108193      1.61  
test  430412      1.53  
      443497      1.53  

[5 rows x 973 columns]

In [ ]:
x['hash_numeric'] = x.apply(lambda x: hash(tuple(x[:-4])), axis=1)

In [12]:
x.head()

L0_S0_F0  L0_S0_F10  L0_S0_F12  L0_S0_F14  L0_S0_F16  L0_S0_F18  \
      Id                                                                        
test  510783    -0.180     -0.066      0.022      0.128     -0.204     -0.147   
train 651542    -0.180     -0.021      0.022      0.168     -0.220     -0.163   
      108193    -0.193     -0.021      0.030      0.208     -0.266     -0.078   
test  430412    -0.153     -0.066      0.022      0.168     -0.056      0.029   
      443497    -0.297     -0.021      0.052      0.208     -0.128     -0.051   

              L0_S0_F2  L0_S0_F20  L0_S0_F22  L0_S0_F4      ...       \
      Id                                                    ...        
test  510783    -0.213     -0.340     -0.340     0.367      ...        
train 651542    -0.250     -0.340     -0.340     0.367      ...        
      108193    -0.272     -0.340     -0.340     0.385      ...        
test  430412    -0.235     -0.338     -0.338     0.385      ...        
      443497    -0.287     -0.340     -0.340     0.367      ...        

              L3_S51_F4256  L3_S51_F4258  L3_S51_F4260  L3_S51_F4262  \
      Id                                                               
test  510783           NaN           NaN           NaN           NaN   
train 651542           NaN           NaN           NaN           NaN   
      108193           NaN           NaN           NaN           NaN   
test  430412           NaN           NaN           NaN           NaN   
      443497           NaN           NaN           NaN           NaN   

              Response  station_start  station_end  time_start  time_end  \
      Id                                                                   
test  510783       NaN            0.0         52.0        0.00      1.61   
train 651542       0.0            0.0         52.0        0.00      1.53   
      108193       0.0            0.0         52.0        0.01      1.61   
test  430412       NaN            0.0         52.0        0.01      1.53   
      443497       NaN            0.0         52.0        0.01      1.53   

              hash_numeric  
      Id                    
test  510783           NaN  
train 651542           NaN  
      108193           NaN  
test  430412           NaN  
      443497           NaN  

[5 rows x 974 columns]